In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os

os.chdir('/content/gdrive/My Drive/Kaggle')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import time, datetime

np.random.seed(2)

sns.set(style='white', context='notebook', palette='deep')

In [0]:
train = pd.read_csv('Dataset/train.csv')
test = pd.read_csv('Dataset/test.csv')

Y_train = train['label']
X_train = train.drop(labels=['label'], axis=1)

del train

g = sns.countplot(Y_train)

Y_train.value_counts()

In [0]:
X_train.isnull().any().describe()

In [0]:
test.isnull().any().describe()

In [0]:
X_train /= 255.0
test /= 255.0

In [0]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [0]:
Y_train = to_categorical(Y_train, num_classes=10)

In [0]:
RANDOM_SEED = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=RANDOM_SEED)

In [0]:
import keras.backend as K
K.clear_session()

dense_layers = [0, 1, 2]
layer_sizes = [64, 128]
conv_layers = [2]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
          NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
          logdir = os.path.join("logs", NAME)

          model = Sequential()
          model.add(Conv2D(layer_size, (3, 3), activation='relu', input_shape=(28, 28, 1)))
          model.add(MaxPooling2D(pool_size=(2, 2)))
          model.add(Dropout(0.25))
          
          for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(0.25))
                
          model.add(Flatten())
          
          for _ in range(dense_layer):
                model.add(Dense(layer_size), activation='relu')
                model.add(Dropout(0.25))
                
          model.add(Dense(10, activation="softmax"))

          model.compile(loss='categorical_crossentropy',
                          optimizer='RMSprop',
                          metrics=['accuracy'],
                          )
          
          learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
          mc = ModelCheckpoint(filepath='Best Model/{}-best_model.hdf5'.format(NAME), monitor='val_acc', verbose=1, save_best_only=True, mode='max')

          datagen = ImageDataGenerator(
          featurewise_center=False,  
          samplewise_center=False,  
          featurewise_std_normalization=False,  
          samplewise_std_normalization=False,  
          zca_whitening=False,  
          rotation_range=10,  
          zoom_range = 0.1, 
          width_shift_range=0.1,  
          height_shift_range=0.1,  
          horizontal_flip=False,  
          vertical_flip=False)

          datagen.fit(X_train) 
          
          model.fit(X_train, Y_train,
                      batch_size=32,
                      epochs=30,
                      validation_data = (X_val, Y_val),
                      callbacks=[learning_rate_reduction, mc])

In [0]:
model = tf.keras.models.load_model('Best Model/2-conv-128-nodes-0-dense-1564306233-best_model.hdf5')

results = model.predict(test)

results = np.argmax(results,axis = 1)


In [0]:
results = pd.Series(results,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist.csv",index=False)